In [1]:
import pandas as pd
import numpy as np
import itertools
from scipy.spatial import distance

pd.set_option('display.max_rows', 500)

### Create inputs

In [2]:
objects = ['t', 'n', 's', 'p', 'c']

In [3]:
coordinates = {'c': (1,4),
              'n': (1,1),
              'p': (1,4),
              's': (1,2),
              't': (1,1),
              'start': (2,2),
              'table': (4,3)}

### Populate dataframe

In [4]:
def fill_dataframe(objects, objects_at_once=2):
    df = pd.DataFrame(columns=['from_node', 'to_node', 'obj_at_once', 'dist', 'weight_new'])
    # Generate list of combinations w/o duplicates
    nodes = list(itertools.chain(*[itertools.combinations(objects, i+1) for i in range(len(objects))]))
    nodes = [''.join(node) for node in nodes]
    
    node_list = []
    for x in range(0, len(nodes)):
        if len(nodes[x]) <= 2:
            df.loc[x, 'from_node'] = 'z'
            df.loc[x, 'to_node'] = nodes[x]
            df.loc[x, 'obj_at_once'] = len(nodes[x])
    
        for y in range(x, len(nodes)):
            if 1 <= (len(nodes[y]) - len(nodes[x])) <= 2:
                if all(x in nodes[y] for x in nodes[x]):
                    node_list.append((nodes[y], nodes[x]))
    
    for tup in node_list:
        df = df.append({'from_node': tup[1], 'to_node': tup[0], 'obj_at_once': len(tup[0]) - len(tup[1])}, ignore_index=True)
    
    # No constraints for how many objects at once
    if objects_at_once == 2:
        df = df
    elif objects_at_once == 1:
        # Add constraint: Only 1 object at once
        df = df.loc[df['obj_at_once'] == 1]

    # Reset index
    df = df.reset_index(drop=True)
    
    return df

In [69]:
#data = fill_dataframe(objects, objects_at_once=2)

### Calculate distances

In [5]:
def calculate_distances(data):
    for row in range(0, len(data)):
        # Distance = start -> obj 1 -> table
        if data['from_node'][row] == 'z' and len(data['to_node'][row]) == 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row]], 
                        coordinates['table'])
                    )
    
        # Distance = start -> obj 1 -> obj 2 -> table
        elif data['from_node'][row] == 'z' and len(data['to_node'][row]) != 1:
            data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['start'], 
                        coordinates[data['to_node'][row][0]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][0]], 
                        coordinates[data['to_node'][row][1]]) +
                    distance.euclidean(
                        coordinates[data['to_node'][row][1]], 
                        coordinates['table'])
                    )
    # Distance = table -> obj 1 -> (obj 2) -> table
        else:
            # Get difference between sequences (from_node, to_node)
            diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
            if len(diff) == 1:
                data.loc[row, 'dist'] = distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) * 2
        
            elif len(diff) == 2:
                data.loc[row, 'dist'] = (distance.euclidean(
                        coordinates['table'], 
                        coordinates[diff[0]]) +
                    distance.euclidean(
                        coordinates[diff[0]], 
                        coordinates[diff[1]]) +
                    distance.euclidean(
                        coordinates[diff[1]], 
                        coordinates['table'])
                    )
    return data

In [71]:
#data = calculate_distances(data)
#data

In [6]:
c0 = {'c': 1.0,
    'n': 1.0,
    'p': 1.0,
    's': 1.0,
    't': 1.0}

k0 = {'c': 1.0,
    'n': 1.0,
    'p': 1.0,
    's': 1.0,
    't': 1.0}

In [7]:
def calculate_edge_weights_params(data, objects, c, k):
    # Reset weights according to weight parameters
    for row in range(0, len(data)):
        diff = [x for x in data['to_node'][row] if x not in data['from_node'][row]]
        
        for obj in objects:
            if len(diff) == 1 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                
            elif len(diff) == 2 and obj in diff:
                data.loc[row, 'weight_new'] = (data['dist'][row] ** k[diff[0]]) * c[diff[0]]
                data.loc[row, 'weight_new'] = (data['weight_new'][row] ** k[diff[1]]) * c[diff[1]]
                
    return data

### Dijkstra + graph classes

In [8]:
class Node:
    def __init__(self, label):
        self.label = label

class Edge:
    def __init__(self, to_node, length):
        self.to_node = to_node
        self.length = length


class Graph:
    def __init__(self):
        self.nodes = set()
        self.edges = dict()

    def add_node(self, node):
        self.nodes.add(node)

    def add_edge(self, from_node, to_node, length):
        edge = Edge(to_node, length)
        if from_node.label in self.edges:
            from_node_edges = self.edges[from_node.label]
        else:
            self.edges[from_node.label] = dict()
            from_node_edges = self.edges[from_node.label]
        from_node_edges[to_node.label] = edge


def min_dist(q, dist):
    """
    Returns the node with the smallest distance in q.
    Implemented to keep the main algorithm clean.
    """
    min_node = None
    for node in q:
        if min_node == None:
            min_node = node
        elif dist[node] < dist[min_node]:
            min_node = node

    return min_node

def dijkstra(graph, source):
    q = set()
    dist = {}
    prev = {}

    for v in graph.nodes:           # initialization
        dist[v] = float('inf')      # unknown distance from source to v
        prev[v] = float('inf')      # previous node in optimal path from source
        q.add(v)                    # all nodes initially in q (unvisited nodes)

    # distance from source to source
    dist[source] = 0

    while q:
        # node with the least distance selected first
        u = min_dist(q, dist)

        q.remove(u)

        if u.label in graph.edges:
            for _, v in graph.edges[u.label].items():
                alt = dist[u] + v.length
                if alt < dist[v.to_node]:
                    # a shorter path to v has been found
                    dist[v.to_node] = alt
                    prev[v.to_node] = u

    return dist, prev


def to_array(prev, from_node):
    """Creates an ordered list of labels as a route."""
    previous_node = prev[from_node]
    route = [from_node.label]
    while previous_node != float('inf'):
        route.append(previous_node.label)
        temp = previous_node
        previous_node = prev[temp]

    route.reverse()
    return route

### Create nodes + edges

In [9]:
def create_nodes_edges(graph, data):
    # Create sorted set of nodes from to_nodes and from_nodes
    nodes = sorted(pd.unique(data[['from_node', 'to_node']].values.ravel()))

    # Create nodes from set
    for x in range(0, len(nodes)):
        globals()[nodes[x]] = Node(nodes[x])    # Create node as global variable
        graph.add_node(globals()[nodes[x]])     # Add node to graph

    # Add edge for row in csv -> from_node, to_node, weight
    for row in range(0, len(data)):
        to_node = globals()[data['to_node'][row]]
        from_node = globals()[data['from_node'][row]]
        graph.add_edge(from_node, to_node, data['weight_new'][row])
    
    return graph

In [341]:
#create_nodes_edges(graph, data)

### Print function

In [10]:
def print_result(dist, prev, start, target):
    print("The quickest path from {} to {} is [{}] with a distance of {}".format(
        start.label,
        target.label,
        " -> ".join(to_array(prev, target)),
        str(round(dist[target], 2))
        )
    )

### Define main function

In [25]:
def main():
    graph = Graph()
    data = fill_dataframe(objects, objects_at_once=1)
    data = calculate_distances(data)
    data = calculate_edge_weights_params(data, objects, c1, k1)
    create_nodes_edges(graph, data)
    dist, prev = dijkstra(graph, z)
    print_result(dist, prev, z, tnspc)

In [26]:
main()

The quickest path from z to tnspc is [z -> s -> ts -> tsp -> tnsp -> tnspc] with a distance of 29.12


In [23]:
c1 = {'c': 1.2,
    'n': 1.0,
    'p': 1.2,
    's': 1.2,
    't': 1.0}

In [24]:
k1 = {'c': 1.0,
    'n': 0.95,
    'p': 0.95,
    's': 1.0,
    't': 0.9}

In [28]:
data2 = fill_dataframe(objects, objects_at_once=1)
data2 = calculate_distances(data2)
data2 = calculate_edge_weights_params(data2, objects, c1, k1)
data2

,from_node,to_node,obj_at_once,dist,weight_new
0,z,t,1,5.01976,3.63535
1,z,n,1,5.01976,4.27184
2,z,s,1,4.16228,4.99473
3,z,p,1,5.39835,5.0304
4,z,c,1,5.39835,6.47801
5,t,tn,1,7.2111,5.91836
6,t,ts,1,6.32456,7.58947
7,t,tp,1,6.32456,5.75515
8,t,tc,1,6.32456,7.58947
9,n,tn,1,7.2111,4.85737
